## '股票大池435_ths.csv'布林带日内交易策略_回测20191230~20220527
### 设置操作标的：context.stockpool = [ '股票大池435_ths.csv' ]

In [ ]:
from tick_trade_api import TradeAPI
# 输入指定账户
trade_api=TradeAPI('79855043',order_policy=MarketPolicy)


In [2]:
source_code="""

# 初始化函数
def init(context):
    #导入模型
    import pandas as pd
    #设置Boll模型参数
    context.timeperiod = 16 
    context.nbdevup = 2 
    context.nbdevdn = 2    
    # 运作周期
    context.runstep = 3
    # 设置基准收益
    #set_benchmark('000300.SH')
    #set_benchmark(002060.SZ')
    # 设置操作标的：
    #context.stockpool = ['002625.SZ', '600657.SH', '002060.SZ', '600359.SH']
    #context.stockpool = ['002060.SZ']
    file_path = '股票大池435_ths.csv'
    context.stockpool = []
    context.stockpool_1 = []
    df =pd.read_csv(file_path)
    context.stockpool_1 = list(df)[1:]
    print('len(context.stockpool_1)=', len(context.stockpool_1))
    context.n = 30 # 最大持股数
    #运用I问财进行股票筛选
    get_iwencai('未停牌,上市时间超过2年')
    
    # 设置手续费
    set_commission(PerShare(type='stock',cost=0.00025))
    #设置可变滑点1%，表示买入价为实际价格乘101%，卖出价为实际价格乘99%
    set_slippage(PriceSlippage(0.01))
    # 固定盈利
    context.win = 25 #盈利几个点卖出
    # 单笔买入仓位
    context.buypct = 0.003
    # 止盈
    context.stopwin = 100
    # 止损
    context.stoploss = 0.13  

    # 设置初识持仓
    #set_holding_stocks(dict(zip(context.stockpool,[702200]*len(context.stockpool))))
    set_holding_stocks(dict(zip(context.stockpool,[0]*len(context.stockpool))))
    
    # 记录日内买入日期时间
    context.stocktrade = {}
    for stock in context.stockpool:
        context.stocktrade[stock] = []
    
    # 获取当前持仓
    holdstock = list(context.portfolio.stock_account.positions.keys())
    print('持仓股票holdstock：', holdstock) 
    # 查询持仓情况
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value
    stocklist = list(context.portfolio.stock_account.positions)
    print('当前持仓市值:', market_value, '持仓股票stocklist:', stocklist, '账户总资产:', int(total_value))
    

# 每个交易日开市前，打印出目前的账户信息.
def before_trading(context):
    stocks=context.iwencai_securities
    stock_t = []
    for security in stocks:
        if security in context.stockpool_1:
            stock_t.append(security)
    context.stockpool = list(set(stock_t))[::]
    print('len(context.stockpool)=', len(context.stockpool))
    #log.info(context.portfolio)
    
    # 记录日内买入日期时间
    context.stocktrade = {}
    for stock in context.stockpool:
        context.stocktrade[stock] = []    
    
    
    
def handle_bar(context, bar_dict):    
    # 导入Python-talib库
    import talib
    # 获取当前分钟
    date = get_datetime().strftime('%Y-%m-%d %H:%M:%S')
    time1 = get_datetime().strftime('%M')
    time = get_datetime().strftime('%M')

    # 一、判断盈亏条件
    # 1.1 止盈止损
    # 获取当前持仓
    holdstock = list(context.portfolio.stock_account.positions.keys())
    for stock in holdstock:
        # 获取数据
        df = history(stock, ['close'], 2, '1m', True, 'pre', is_panel=1)
        new_price = df['close'].iloc[-1]
        # 获取成本价
        cost_basis = context.portfolio.positions[stock].cost_basis

        # 获取可卖股数等持仓情况
        amount = context.portfolio.positions[stock].amount
        available_amount = context.portfolio.positions[stock].available_amount
        market_value = context.portfolio.stock_account.market_value
        total_value = context.portfolio.stock_account.total_value
        stocklist = list(context.portfolio.stock_account.positions)
        
        
        if new_price/cost_basis-1>context.stopwin and available_amount>0:
            # 卖出所有股票
            print("止盈卖出 %s" % (stock), available_amount, '股')
            order(stock,-available_amount)
            # 移除本股票本轮交易时间记录
            context.stocktrade[stock] = []

        elif new_price/cost_basis-1< -context.stoploss and available_amount>0:
            # 卖出该股全部
            print("止损卖出 %s" % (stock), available_amount, '股')
            order(stock,-available_amount)
            # 移除本股票本轮交易时间记录
            context.stocktrade[stock] = []
    
            amount = context.portfolio.positions[stock].amount
            available_amount = context.portfolio.positions[stock].available_amount
            market_value = context.portfolio.stock_account.market_value
            total_value = context.portfolio.stock_account.total_value
            stocklist = list(context.portfolio.stock_account.positions)
            print('持仓股数:', amount, '可卖股数:', available_amount, '持仓股票', stocklist, '账户总资产:', int(total_value))
            
    # 1.2 达到获利目标后回落卖出
    # 获取当前持仓
    holdstock = list(context.portfolio.stock_account.positions.keys())
    for stock in holdstock:
        # 获取数据
        df = history(stock, ['close'], 2, '1m', True, 'pre', is_panel=1)
        new_price = df['close'].iloc[-1]
        # 获取成本价
        cost_basis = context.portfolio.positions[stock].cost_basis

        # 获取可卖股数等持仓情况
        available_amount = context.portfolio.positions[stock].available_amount
        market_value = context.portfolio.stock_account.market_value
        total_value = context.portfolio.stock_account.total_value
        # 取交易时间数据
        #print('context.stocktrade[stock]:', context.stocktrade[stock])


        if len(context.stocktrade[stock])>0:
            d1 = context.stocktrade[stock][0]
            dd1 = d1[:4]+d1[5:7]+d1[8:10]+' '+d1[11:16]
            dd2 = date

            df = get_price(stock, dd1, dd2,'1m', ['close', 'open', 'low', 'high'], skip_paused = False, fq = 'pre', is_panel = 1)
            new_price = df['close'].iloc[-1]

            # 计算本轮首次买入后至最新价期间的最高价及最大涨幅%
            max_price = df['high'].max()
            max_zf = (max_price-cost_basis)/cost_basis*100
            # 当最大涨幅大于context.win时候
            if max_zf>context.win and max_zf<40:
                # 计算本轮首次买入后至最新价期间的冲高回落幅度%
                hlfd = ((max_price-new_price)/(max_price-cost_basis))*100
                if hlfd > 20 and available_amount>0:
                    # 卖出该股全部
                    print("回落卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                    order(stock,-available_amount)
                    # 移除本股票本轮交易时间记录
                    context.stocktrade[stock] = []
                    
                    amount = context.portfolio.positions[stock].amount
                    available_amount = context.portfolio.positions[stock].available_amount
                    market_value = context.portfolio.stock_account.market_value
                    total_value = context.portfolio.stock_account.total_value
                    stocklist = list(context.portfolio.stock_account.positions)
                    print('持仓股数:', amount, '可卖股数:', available_amount, '持仓股票列表', stocklist, '账户总资产:', int(total_value))

            # 当最大涨幅大于等于40%时候
            if max_zf>=40:
                # 计算本轮首次买入后至最新价期间的冲高回落幅度%
                hlfd = ((max_price-new_price)/(max_price-cost_basis))*100
                if hlfd > 10 and available_amount>0:
                    # 卖出该股全部
                    print("回落卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                    order(stock,-available_amount)
                    # 移除本股票本轮交易时间记录
                    context.stocktrade[stock] = []
                    
                    amount = context.portfolio.positions[stock].amount
                    available_amount = context.portfolio.positions[stock].available_amount
                    market_value = context.portfolio.stock_account.market_value
                    total_value = context.portfolio.stock_account.total_value
                    stocklist = list(context.portfolio.stock_account.positions)
                    print('持仓股数:', amount, '可卖股数:', available_amount, '持仓股票列表', stocklist, '账户总资产:', int(total_value))


    
    # 二、布林带日内交易策略
    # 循环股票
    #print('context.stockpool:', context.stockpool)
    for stock in context.stockpool:
        # 获取成本价
        cost_basis = context.portfolio.positions[stock].cost_basis

        # 获取数据
        df = history(stock, ['close','high','low'], 2400,'1m', True, 'pre', is_panel=1)
        # 提取数据
        close_price = df['close']
        high_price = df['high']
        low_price = df['low']
        new_price = df['close'].iloc[-1]
        
        # 导入Python-talib库
        import talib
        import numpy as np
        # 计算指标，获取BOLL值
        available_amount = context.portfolio.positions[stock].available_amount
        upperband, middleband, lowerband = talib.BBANDS(close_price.values, timeperiod=context.timeperiod, nbdevup=context.nbdevup, nbdevdn=context.nbdevdn)
        T_value = upperband-lowerband
        T_value = T_value[19:]
        mean = np.mean(T_value)
        new = T_value[-1]
        
        if T_value[-2] >T_value[-1] and new<mean and new_price<lowerband[-1]:            
            order_percent(stock, context.buypct)
            total_value = context.portfolio.stock_account.total_value
            #print(stock, '当前处于相对低点{}'.format(new_price),'买入股票', '账户总资产:', int(total_value))
            # 记录这次买入时间
            context.stocktrade[stock].append(date)

        elif available_amount>0 and T_value[-2] >T_value[-1] and new<mean and new_price>upperband[-1]:
            print(stock,'当前处于相对高点{}'.format(new_price))
            order_percent(stock, -context.buypct)
            # 记录这次卖出时间
            context.stocktrade[stock].append(date)
            market_value = context.portfolio.stock_account.market_value
            total_value = context.portfolio.stock_account.total_value
            print(stock,"布林相对高点卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
            print('持仓股票市值', market_value ,'账户总资产:', int(total_value))
            # 移除本股票本轮交易时间记录
            context.stocktrade[stock] = []
            
# 收盘后运行函数,用于储存自定义参数、全局变量,执行盘后选股等
def after_trading(context):
    
    log.info('一天结束')            

"""

# # 系统重置
# try:
#     import shutil
#     shutil.rmtree('persist/研究环境策略')
# except:
#     pass

# # 回测运行
# # research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
# research_strategy(source_code, start_date='20191230', end_date='20220527', capital_base=float(10000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)




In [3]:
# 系统重置
try:
    import shutil
    shutil.rmtree('persist/研究环境策略')
except:
    pass


In [ ]:
# 模拟运行
#rtrade = research_trade('研究环境策略', source_code, capital_base=float(4550000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易



In [ ]:
# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
#research_strategy(source_code, start_date='20191230', end_date='20220527', capital_base=float(4551000), frequency='MINUTE', stock_market='STOCK', benchmark=None)


In [4]:
# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
research_strategy(source_code, start_date='20191230', end_date='20220527', capital_base=float(10000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)


2019-12-30 00:00:00.000000 - INFO - len(context.stockpool_1)= 435
2019-12-30 00:00:00.000000 - INFO - 持仓股票holdstock： []
2019-12-30 00:00:00.000000 - INFO - 当前持仓市值: 0 持仓股票stocklist: [] 账户总资产: 10000000
2019-12-30 09:00:00.000000 - INFO - len(context.stockpool)= 305
2019-12-30 09:00:00.000000 - INFO - Portfolio({'portfolio_value': 10000000.0, 'frozen_cash': 0, 'market_value': 0, 'returns': 0.0, 'positions': [], 'starting_value': 10000000.0, 'datetime': datetime.datetime(2019, 12, 30, 9, 0), 'pnl': 0.0, 'start_date': datetime.date(2019, 12, 30), 'available_cash': 10000000.0})
2019-12-30 10:17:00.000000 - WARN - 下单失败: [603598.SH] 已涨停。
2019-12-30 10:31:00.000000 - WARN - 下单失败: [002269.SZ] 已涨停。
2019-12-30 13:17:00.000000 - WARN - 下单失败: [603598.SH] 已涨停。
2019-12-30 13:47:00.000000 - WARN - 下单失败: [603598.SH] 已涨停。
2019-12-30 14:35:00.000000 - WARN - 下单失败: [002269.SZ] 已涨停。
2019-12-30 14:44:00.000000 - WARN - 下单失败: [603598.SH] 已涨停。
2019-12-30 15:30:00.000000 - INFO - 一天结束
2019-12-31 09:00:00.000000

2020-01-02 09:46:00.000000 - WARN - 下单失败: [603598.SH] 已涨停。
2020-01-02 09:53:00.000000 - INFO - 000729.SZ 当前处于相对高点6.52
2020-01-02 09:53:00.000000 - INFO - 000729.SZ 布林相对高点卖出:000729.SZ,卖出价格:6.52,毛盈利0.09%
2020-01-02 09:53:00.000000 - INFO - 持仓股票市值 5986374.0 账户总资产: 10064546
2020-01-02 09:54:00.000000 - INFO - 600237.SH 当前处于相对高点3.59
2020-01-02 09:54:00.000000 - INFO - 600237.SH 布林相对高点卖出:600237.SH,卖出价格:3.59,毛盈利0.46%
2020-01-02 09:54:00.000000 - INFO - 持仓股票市值 5958561.0 账户总资产: 10066701
2020-01-02 09:57:00.000000 - INFO - 601118.SH 当前处于相对高点4.95
2020-01-02 09:57:00.000000 - INFO - 601118.SH 布林相对高点卖出:601118.SH,卖出价格:4.95,毛盈利-0.01%
2020-01-02 09:57:00.000000 - INFO - 持仓股票市值 5930095.0 账户总资产: 10068241
2020-01-02 10:05:00.000000 - INFO - 300253.SZ 当前处于相对高点15.17
2020-01-02 10:05:00.000000 - INFO - 300253.SZ 布林相对高点卖出:300253.SZ,卖出价格:15.17,毛盈利1.51%
2020-01-02 10:05:00.000000 - INFO - 持仓股票市值 5965705.0 账户总资产: 10076335
2020-01-02 10:10:00.000000 - INFO - 600435.SH 当前处于相对高点8.41
2020-01-02 10:10:00.000000 - IN

2020-01-02 13:52:00.000000 - INFO - 600292.SH 布林相对高点卖出:600292.SH,卖出价格:5.74,毛盈利2.14%
2020-01-02 13:52:00.000000 - INFO - 持仓股票市值 6687378.0 账户总资产: 10099594
2020-01-02 13:52:00.000000 - INFO - 601186.SH 当前处于相对高点10.44
2020-01-02 13:52:00.000000 - INFO - 601186.SH 布林相对高点卖出:601186.SH,卖出价格:10.44,毛盈利3.54%
2020-01-02 13:52:00.000000 - INFO - 持仓股票市值 6657102.0 账户总资产: 10099376
2020-01-02 13:53:00.000000 - INFO - 002796.SZ 当前处于相对高点37.54
2020-01-02 13:53:00.000000 - INFO - 002796.SZ 布林相对高点卖出:002796.SZ,卖出价格:37.54,毛盈利5.73%
2020-01-02 13:53:00.000000 - INFO - 持仓股票市值 6656092.0 账户总资产: 10098099
2020-01-02 13:56:00.000000 - INFO - 002371.SZ 当前处于相对高点92.25
2020-01-02 13:56:00.000000 - INFO - 002371.SZ 布林相对高点卖出:002371.SZ,卖出价格:92.25,毛盈利1.98%
2020-01-02 13:56:00.000000 - INFO - 持仓股票市值 6658611.0 账户总资产: 10098060
2020-01-02 14:01:00.000000 - INFO - 300082.SZ 当前处于相对高点6.6
2020-01-02 14:01:00.000000 - INFO - 300082.SZ 布林相对高点卖出:300082.SZ,卖出价格:6.6,毛盈利9.36%
2020-01-02 14:01:00.000000 - INFO - 持仓股票市值 6691393.0 账户总资产: 1010

2020-01-03 10:24:00.000000 - INFO - 持仓股票市值 7268147.0 账户总资产: 10116835
2020-01-03 10:26:00.000000 - INFO - 002073.SZ 当前处于相对高点4.88
2020-01-03 10:26:00.000000 - INFO - 002073.SZ 布林相对高点卖出:002073.SZ,卖出价格:4.88,毛盈利2.74%
2020-01-03 10:26:00.000000 - INFO - 持仓股票市值 7271058.0 账户总资产: 10119674
2020-01-03 10:38:00.000000 - INFO - 002062.SZ 当前处于相对高点3.55
2020-01-03 10:38:00.000000 - INFO - 002062.SZ 布林相对高点卖出:002062.SZ,卖出价格:3.55,毛盈利0.61%
2020-01-03 10:38:00.000000 - INFO - 持仓股票市值 7440313.0 账户总资产: 10113786
2020-01-03 10:40:00.000000 - INFO - 600178.SH 当前处于相对高点4.87
2020-01-03 10:40:00.000000 - INFO - 600178.SH 布林相对高点卖出:600178.SH,卖出价格:4.87,毛盈利0.5%
2020-01-03 10:40:00.000000 - INFO - 持仓股票市值 7465702.0 账户总资产: 10109575
2020-01-03 10:52:00.000000 - INFO - 600309.SH 当前处于相对高点57.62
2020-01-03 10:52:00.000000 - INFO - 600309.SH 布林相对高点卖出:600309.SH,卖出价格:57.62,毛盈利1.28%
2020-01-03 10:52:00.000000 - INFO - 持仓股票市值 7767428.0 账户总资产: 10087707
2020-01-03 10:54:00.000000 - INFO - 600577.SH 当前处于相对高点2.77
2020-01-03 10:54:00.000

2020-01-03 14:33:00.000000 - INFO - 持仓股票市值 8630435.0 账户总资产: 10117866
2020-01-03 14:49:00.000000 - INFO - 600030.SH 当前处于相对高点25.31
2020-01-03 14:49:00.000000 - INFO - 600030.SH 布林相对高点卖出:600030.SH,卖出价格:25.31,毛盈利-1.22%
2020-01-03 14:49:00.000000 - INFO - 持仓股票市值 8764685.0 账户总资产: 10101349
2020-01-03 14:53:00.000000 - INFO - 600051.SH 当前处于相对高点6.05
2020-01-03 14:53:00.000000 - INFO - 600051.SH 布林相对高点卖出:600051.SH,卖出价格:6.05,毛盈利-0.52%
2020-01-03 14:53:00.000000 - INFO - 持仓股票市值 8855232.0 账户总资产: 10100998
2020-01-03 14:56:00.000000 - INFO - 002761.SZ 当前处于相对高点11.24
2020-01-03 14:56:00.000000 - INFO - 002761.SZ 布林相对高点卖出:002761.SZ,卖出价格:11.24,毛盈利0.58%
2020-01-03 14:56:00.000000 - INFO - 持仓股票市值 8858418.0 账户总资产: 10103307
2020-01-03 14:57:00.000000 - INFO - 000682.SZ 当前处于相对高点5.2
2020-01-03 14:57:00.000000 - INFO - 000682.SZ 布林相对高点卖出:000682.SZ,卖出价格:5.2,毛盈利2.22%
2020-01-03 14:57:00.000000 - INFO - 持仓股票市值 8889681.0 账户总资产: 10104655
2020-01-03 15:00:00.000000 - WARN - 订单取消: 开盘价交易模式下，使用分钟交易时，15点的订单不予下单
2020-01-0

...

2020-01-08 10:41:00.000000 - INFO - 持仓股票市值 10094329.0 账户总资产: 10241540
2020-01-08 10:46:00.000000 - INFO - 002371.SZ 当前处于相对高点89.34
2020-01-08 10:46:00.000000 - INFO - 002371.SZ 布林相对高点卖出:002371.SZ,卖出价格:89.34,毛盈利-1.35%
2020-01-08 10:46:00.000000 - INFO - 持仓股票市值 10098691.0 账户总资产: 10241974
2020-01-08 10:47:00.000000 - INFO - 300137.SZ 当前处于相对高点7.82
2020-01-08 10:47:00.000000 - INFO - 300137.SZ 布林相对高点卖出:300137.SZ,卖出价格:7.82,毛盈利1.02%
2020-01-08 10:47:00.000000 - INFO - 持仓股票市值 10126694.0 账户总资产: 10238486
2020-01-08 10:49:00.000000 - INFO - 603987.SH 当前处于相对高点9.79
2020-01-08 10:49:00.000000 - INFO - 603987.SH 布林相对高点卖出:603987.SH,卖出价格:9.79,毛盈利1.65%
2020-01-08 10:49:00.000000 - INFO - 持仓股票市值 10126301.0 账户总资产: 10238220
2020-01-08 10:59:00.000000 - INFO - 002311.SZ 当前处于相对高点37.69
2020-01-08 10:59:00.000000 - INFO - 002311.SZ 布林相对高点卖出:002311.SZ,卖出价格:37.69,毛盈利3.83%
2020-01-08 10:59:00.000000 - INFO - 持仓股票市值 10240343.0 账户总资产: 10270806
2020-01-08 11:00:00.000000 - INFO - 603612.SH 当前处于相对高点12.17
2020-01-08 11

2020-01-08 14:17:00.000000 - WARN - 300430.SZ 下单股数为0,订单无效.
2020-01-08 14:18:00.000000 - WARN - 002714.SZ 下单股数为0,订单无效.
2020-01-08 14:18:00.000000 - WARN - 002151.SZ 下单股数为0,订单无效.
2020-01-08 14:18:00.000000 - WARN - 300716.SZ 下单股数为0,订单无效.
2020-01-08 14:19:00.000000 - WARN - 000887.SZ 下单股数为0,订单无效.
2020-01-08 14:19:00.000000 - WARN - 300082.SZ 下单股数为0,订单无效.
2020-01-08 14:21:00.000000 - WARN - 000818.SZ 下单股数为0,订单无效.
2020-01-08 14:21:00.000000 - WARN - 601801.SH 下单股数为0,订单无效.
2020-01-08 14:22:00.000000 - WARN - 300015.SZ 下单股数为0,订单无效.
2020-01-08 14:24:00.000000 - WARN - 002615.SZ 下单股数为0,订单无效.
2020-01-08 14:39:00.000000 - WARN - 002371.SZ 下单股数为0,订单无效.
2020-01-08 14:40:00.000000 - INFO - 300699.SZ 当前处于相对高点45.39
2020-01-08 14:40:00.000000 - INFO - 300699.SZ 布林相对高点卖出:300699.SZ,卖出价格:45.39,毛盈利-1.33%
2020-01-08 14:40:00.000000 - INFO - 持仓股票市值 10134411.0 账户总资产: 10161675
2020-01-08 14:41:00.000000 - WARN - 下单失败: [000887.SZ] 已涨停。
2020-01-08 14:41:00.000000 - INFO - 603098.SH 当前处于相对高点10.77
2020-01-08 14:41

2020-01-09 10:29:00.000000 - INFO - 300082.SZ 当前处于相对高点6.67
2020-01-09 10:29:00.000000 - INFO - 300082.SZ 布林相对高点卖出:300082.SZ,卖出价格:6.67,毛盈利6.29%
2020-01-09 10:29:00.000000 - INFO - 持仓股票市值 10245563.0 账户总资产: 10276175
2020-01-09 10:32:00.000000 - WARN - 002572.SZ 下单股数为0,订单无效.
2020-01-09 10:32:00.000000 - WARN - 300315.SZ 下单股数为0,订单无效.
2020-01-09 10:35:00.000000 - INFO - 300511.SZ 当前处于相对高点7.03
2020-01-09 10:35:00.000000 - INFO - 300511.SZ 布林相对高点卖出:300511.SZ,卖出价格:7.03,毛盈利1.97%
2020-01-09 10:35:00.000000 - INFO - 持仓股票市值 10237542.0 账户总资产: 10267797
2020-01-09 10:37:00.000000 - INFO - 300547.SZ 当前处于相对高点10.0
2020-01-09 10:37:00.000000 - INFO - 300547.SZ 布林相对高点卖出:300547.SZ,卖出价格:10.0,毛盈利3.12%
2020-01-09 10:37:00.000000 - INFO - 持仓股票市值 10228765.0 账户总资产: 10259000
2020-01-09 10:38:00.000000 - WARN - 300418.SZ 下单股数为0,订单无效.
2020-01-09 10:38:00.000000 - WARN - 601588.SH 下单股数为0,订单无效.
2020-01-09 10:39:00.000000 - WARN - 600322.SH 下单股数为0,订单无效.
2020-01-09 10:40:00.000000 - INFO - 603601.SH 当前处于相对高点8.21
2020-01

2020-01-09 13:51:00.000000 - INFO - 300290.SZ 布林相对高点卖出:300290.SZ,卖出价格:6.8,毛盈利0.96%
2020-01-09 13:51:00.000000 - INFO - 持仓股票市值 10070352.0 账户总资产: 10269110
2020-01-09 13:52:00.000000 - INFO - 600415.SH 当前处于相对高点4.02
2020-01-09 13:52:00.000000 - INFO - 600415.SH 布林相对高点卖出:600415.SH,卖出价格:4.02,毛盈利4.14%
2020-01-09 13:52:00.000000 - INFO - 持仓股票市值 10066551.0 账户总资产: 10268097
2020-01-09 14:05:00.000000 - INFO - 002651.SZ 当前处于相对高点4.95
2020-01-09 14:05:00.000000 - INFO - 002651.SZ 布林相对高点卖出:002651.SZ,卖出价格:4.95,毛盈利1.14%
2020-01-09 14:05:00.000000 - INFO - 持仓股票市值 10235571.0 账户总资产: 10266159
2020-01-09 14:06:00.000000 - INFO - 000538.SZ 当前处于相对高点88.91
2020-01-09 14:06:00.000000 - INFO - 000538.SZ 布林相对高点卖出:000538.SZ,卖出价格:88.91,毛盈利-0.96%
2020-01-09 14:06:00.000000 - INFO - 持仓股票市值 10240828.0 账户总资产: 10268232
2020-01-09 14:06:00.000000 - INFO - 600320.SH 当前处于相对高点3.73
2020-01-09 14:06:00.000000 - INFO - 600320.SH 布林相对高点卖出:600320.SH,卖出价格:3.73,毛盈利0.26%
2020-01-09 14:06:00.000000 - INFO - 持仓股票市值 10210242.0 账户总资产: 1

CustomException: Traceback (most recent call last):
  File strategy.py, line 145 in handle_bar
    
    --> cost_basis = 11.188062480538084
    --> new_price = 10.7
    --> amount = 4600
    --> time = '31'
    --> context = Context({stoploss = 0.13, stockpool_1 = ['300847.SZ', '000705.SZ', '301102.SZ', '605303.SH', '300937.SZ', '301113.SZ', '300511.SZ', '600866.SH', '603867.SH', '6 ...
    --> stocklist = ['002270.SZ', '300667.SZ', '600276.SH', '300547.SZ', '000031.SZ', '600880.SH', '601992.SH', '000800.SZ', '603606.SH', '600312.SH', '300511.SZ', '600156.SH', '60 ...
    --> holdstock = ['002270.SZ', '300667.SZ', '600276.SH', '300547.SZ', '000031.SZ', '600880.SH', '601992.SH', '000800.SZ', '603606.SH', '600312.SH', '300511.SZ', '600156.SH', '60 ...
    --> stock = '002761.SZ'
    --> market_value = 10287494.0
    --> date = '2020-01-10 09:31:00'
    --> talib = <module 'talib' from '/opt/conda/lib/python3.5/site-packages/talib/__init__.py'>
    --> time1 = '31'
    --> available_amoun